Acquire transaction dataset
==============


I start this project by searching for some online open dataset related to bank transaction. Unfortunately, I was not able find an available dataset ready to use. It was not a suprise given that bank transactions are highly sensitive data. Therefore, as a starting point, I decided to used my own bank transactions from the last two years. First of all, I need to find a secure way to extract my transaction programmaly and can be easily automated. Here are some options I found through internet.


**Third Party Aggregator - developer API：**
There are a number of bank/investment account data providers, these company request bank transaction data on behalf of their users. Most of these providers untilize some combination of direct bank access (for large institutions) and screen scraping (for institution don't have or provide directly access). The services I looked into were Yodlee, Intuit, MX and Plaid. Among which, Yodlee apears to mostly widely used provider. It provides a free developer sandbox access with update to 5 live users. Plaid, on the other hand, is a newer provider. The also provide free developer access, with connections available for up to 100 live accounts, which certainly sounds more attractive than Yodlee. The other thing I like about Plaid is that they have great documentation and provide a nice interface to Python. Intuit used to privide similar aggregation servers but it discontinued its Financial Data APIs (formerly known as CAD) in 2016. MX provides similar aggregation services, I was not able to find a free developer access from the website.


**Open Financial Exchange(OFX) API：**
OFX is an open standard for client-server systems and cloud based APIs for exchanging financial data. According to the main OFX site(http://www.ofx.net/), "The specification is freely licensed, allowing any software developer to design an interface that will be supported on the front-end." There are a few Python script/tool to do this, basically, you will find the right configure of your bank and download the financial data in OFX format(simliar to XML) and then parse the file. I didn't try this option out. Here is a available tutorial https://thefinancebuff.com/replacing-microsoft-money-part-5-ofx-scripts.html.



Implemetation with Plaid
---
I decided to go with Plaid API as it has a nice free developer access and nice technique documentations. The following section  demonstrated an approach to extract your bank transactions programmably using Python. In order to use Plaied, you  need to create an account with Plaid (https://plaid.com/). Once you have an account, choose 'Build in the Plaid Development Environment' and apply a developer API key, this may take several days to be approved. From Plaid's documentation, the development environment supports live credentials and data for every institution Plaid supports. The maximum amount of unique live credential sets that may be linked is 100. credential sets refers to the login credential you use to login to your bank's online banking site.

The figure below represent a high level flow of how this worked. Plaid will extract bank transactions from your banks. Once Plaid verified a bank connection with you own online banking credential for one of your bank, then you can use a unique access token, along with the API key to download transaction for that institution. You can use the same process to download transaction from other banks that you have an account. After transactions are extracted, I stored them into a collection in MongoDB for further analaysis and modeling.

![title](img/ETL.png)

**Get AccessToken for your bank:**
Follow the quick start guide from Plaid (https://github.com/plaid/quickstart/tree/master/python). Note that you need to updaet your Plaid API keys (client ID, secret, public_key) in server.py. After you have successfully connected a account, go back to your command line window, and copy the access_token

In [ ]:
from plaid import Client

# Available environments are 'sandbox', 'development', and 'production'.
client = Client(client_id='', secret='', public_key='', environment='sandbox')
# assess_token from Plaid Link
access_token=''

**Get transactions with API key and access_token:** Be aware that Plaid will not be able to get all your historical transactions. For example, I couldn't get my transactions in 2015. The amount of data come back differs from bank to bank.

In [95]:
response = client.Transactions.get(access_token, start_date='2013-07-12', end_date='2018-01-19')
transactions = response['transactions']

# the transactions in the response are paginated, so make multiple calls while increasing the offset to
# retrieve all transactions
while len(transactions) < response['total_transactions']:
    response = client.Transactions.get(access_token, start_date='2013-07-12', end_date='2018-01-21',
                                       offset=len(transactions)
                                      )
    transactions.extend(response['transactions'])
    
Category = client.Categories.get()   

Below is an example transaction downloaded. It's a json format document with varies key values pairs that describe a transaction. The data points I'm interested in are name, date, amount. It indicates that I spent $14.15 at 'MR BING VANDERBILT LL' on Jan 19 2018 - It was my lunch. Note that here Plaid also returns the category. It appears for this transaction it could confidently determine the category.

```json
{
    "account_owner": null, 
    "category": null, 
    "account_id": "4R0myMYJNwfbDJqNrnexHV3B35KVQYf0bN389", 
    "pending_transaction_id": "JjNPVdKryLILJzqdgE0vcLdPy79YqYtgwoyqv", 
    "transaction_id": "DyEdbM1kOmhbXaw0Y7gAumQ5Lx8DZRIoDv3nD", 
    "transaction_type": "unresolved", 
    "amount": 14.15, 
    "location": {
        "city": null, 
        "zip": null, 
        "store_number": null, 
        "lon": null, 
        "state": null, 
        "address": null, 
        "lat": null
    }, 
    "payment_meta": {
        "reference_number": null, 
        "payer": null, 
        "payment_method": null, 
        "reason": null, 
        "payee": null, 
        "ppd_id": null, 
        "payment_processor": "Square", 
        "by_order_of": null
    }, 
    "date": "2018-01-19", 
    "category_id": null, 
    "pending": false, 
    "name": "MR BING VANDERBILT LL"
}
```

**Load the data into MongoDB:** finally, I'll dump the transactions into my local Mongo instance. It'll be more convenient to analyse and explore json data with Mongo queries.

In [89]:
##from json import dumps
##print dumps(transactions,indent=4)

from pymongo import MongoClient
mongoclient = MongoClient('localhost')
##Mongoclient.database_names()
db = mongoclient['bankdata']
new_result = db.transactions.insert_many(transactions)
##new_result = db.category.insert_many(Category['categories'])

The data is now in 'transactions' collection under database 'bankdata' in my local mongodb instance. I have around 1600 transactions.It's not a lot of data but it's something enough for me to start this project.